In [2]:
import numpy as np
import cv2
import glob
import numpy as np
import os
import shutil

%matplotlib inline

class StereoCalibration(object):
    def __init__(self, filepath, square_size):
        # termination criteria
        self.criteria = (cv2.TERM_CRITERIA_MAX_ITER +
                         cv2.TERM_CRITERIA_EPS, 30, 0.001)
        self.criteria_cal = (cv2.TERM_CRITERIA_EPS +
                             cv2.TERM_CRITERIA_COUNT, 100, 1e-5)

        # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
        self.objp = np.zeros((9*6, 3), np.float32)
        self.objp[:, :2] = np.mgrid[0:9, 0:6].T.reshape(-1, 2)
        for pt in self.objp:
            pt *= square_size

        # Arrays to store object points and image points from all the images.
        self.objpoints = []  # 3d point in real world space
        self.imgpoints_l = []  # 2d points in image plane.
        self.imgpoints_r = []  # 2d points in image plane.

        self.process_images(filepath)
        self.stereo_calibrate()

    def process_images(self, filepath):
        images_right = glob.glob(os.path.join(filepath, "right/*.png"))
        images_left = glob.glob(os.path.join(filepath, "left/*.png"))
        images_left.sort()
        images_right.sort()
        
        for image_left, image_right in zip(images_left, images_right):
            img_l = cv2.imread(image_left, 0)
            img_r = cv2.imread(image_right, 0)

            # Find the chess board corners
            flags = 0
            flags |= cv2.CALIB_CB_ADAPTIVE_THRESH
            flags |= cv2.CALIB_CB_NORMALIZE_IMAGE
            ret_l, corners_l = cv2.findChessboardCorners(img_l, (9, 6), flags)
            ret_r, corners_r = cv2.findChessboardCorners(img_r, (9, 6), flags)
            
            # if corners are found in both images, refine and add data
            if ret_l and ret_r:
                self.objpoints.append(self.objp)
                rt = cv2.cornerSubPix(img_l, corners_l, (5, 5),
                                      (-1, -1), self.criteria)
                self.imgpoints_l.append(corners_l)
                rt = cv2.cornerSubPix(img_r, corners_r, (5, 5),
                                      (-1, -1), self.criteria)
                self.imgpoints_r.append(corners_r)

            self.img_shape = img_r.shape[::-1]

        init = True
        if init:
            self.M1 = cv2.initCameraMatrix2D(self.objpoints, self.imgpoints_l, self.img_shape, 0);
            self.M2 = cv2.initCameraMatrix2D(self.objpoints, self.imgpoints_r, self.img_shape, 0);
            self.d1 = np.array([0.0, 0.0])
            self.d2 = np.array([0.0, 0.0])
        else:
            rt, self.M1, self.d1, self.r1, self.t1 = cv2.calibrateCamera(
                self.objpoints, self.imgpoints_l, img_shape, None, None)
            rt, self.M2, self.d2, self.r2, self.t2 = cv2.calibrateCamera(
                self.objpoints, self.imgpoints_r, img_shape, None, None)

    def stereo_calibrate(self):
        
        # config
        flags = 0
        flags |= cv2.CALIB_FIX_ASPECT_RATIO
        flags |= cv2.CALIB_USE_INTRINSIC_GUESS
        flags |= cv2.CALIB_SAME_FOCAL_LENGTH
        flags |= cv2.CALIB_ZERO_TANGENT_DIST
        flags |= cv2.CALIB_RATIONAL_MODEL
        flags |= cv2.CALIB_FIX_K3
        flags |= cv2.CALIB_FIX_K4
        flags |= cv2.CALIB_FIX_K5
        flags |= cv2.CALIB_FIX_K6
        stereocalib_criteria = (cv2.TERM_CRITERIA_COUNT +
                                cv2.TERM_CRITERIA_EPS, 100, 1e-5)
        
        # stereo calibration procedure
        ret, self.M1, self.d1, self.M2, self.d2, R, T, E, F = cv2.stereoCalibrate(
            self.objpoints, self.imgpoints_l, self.imgpoints_r,
            self.M1, self.d1, self.M2, self.d2, self.img_shape,
            criteria=stereocalib_criteria, flags=flags)
        
        # construct Homography
        plane_depth = 40
        n = np.array([[0.0], [0.0], [-1.0]])
        d_inv = 1/plane_depth
        self.H = (R - d_inv * np.dot(T, n.transpose()))
        self.H = np.dot(self.M2, np.dot(self.H, np.linalg.inv(self.M1)))
        self.H /= self.H[2, 2]
        
        # rectify Homography for right camera
        disparity = (self.M1[0,0] * 3.5 / plane_depth)
        self.H[0, 2] += disparity
        self.H = np.linalg.inv(self.H) 
    
    def rectify_dataset(self, dataset_dir):
        images_left = glob.glob(dataset_dir + 'left/*.png')
        print(dataset_dir + 'left/*.png')
        images_right = glob.glob(dataset_dir + 'right/*.png')
        left_result_dir = os.path.join(dataset_dir, "Rectify", "left")
        right_result_dir = os.path.join(dataset_dir, "Rectify", "right")
        
        if not os.path.exists(left_result_dir):
            os.makedirs(left_result_dir)
        else:
            shutil.rmtree(left_result_dir)
            os.makedirs(left_result_dir)
        
        if not os.path.exists(right_result_dir):
            os.makedirs(right_result_dir)
        else:
            shutil.rmtree(right_result_dir)
            os.makedirs(right_result_dir)
        
        images_left.sort()
        images_right.sort()
        
        for image_left, image_right in zip(images_left, images_right):
            
            # read images
            img_l = cv2.imread(image_left, 0)
            img_r = cv2.imread(image_right, 0)
            
            # warp right image
            img_r = cv2.warpPerspective(img_r, self.H, img_r.shape[::-1], 
                                        cv2.INTER_LINEAR + cv2.WARP_FILL_OUTLIERS + cv2.WARP_INVERSE_MAP)
            
            # save images
            cv2.imwrite(os.path.join(left_result_dir, os.path.basename(image_left)), img_l)
            cv2.imwrite(os.path.join(right_result_dir, os.path.basename(image_right)), img_r)


### Execution

In [3]:
image_dir = "/home/prowsome/capture_calib/cristina_captures/dataset_v2/"
square_size = 2.35
cal_data = StereoCalibration(image_dir, square_size)
dataset_dir = "/home/prowsome/repositories/dataset/labOv9282Range5mCalibTesting/720p/Uncalib8bit/"
# dataset_dir = "/home/prowsome/capture_calib/cristina_captures/homog/"
cal_data.rectify_dataset(dataset_dir)

/home/prowsome/repositories/dataset/labOv9282Range5mCalibTesting/720p/Uncalib8bit/left/*.png


H [1.001990285402552, 0.001602674879339676, -2.018384410596823;
 -0.000163615872416059, 1.001526280663275, 7.226138400326844;
 1.530658430137132e-06, 1.275590348749002e-06, 1]




